# Sensitivity Analysis: Sestamibi Scan

### Objective

Calculate the **sensitivity of Sestamibi scans** by assessing true positive rates among patients with confirmed disease.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging and surgical outcomes)

### Outputs

- **Sensitivity for Sestamibi scans:** **64.7%**
  - **True Positives:** 112
  - **Total with disease:** 173

---


In [6]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [7]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)


In [8]:
def check_concordance(row):
    imaging = row['Sestamibi']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'

    
    

In [28]:
# Define modality
modality = 'Sestamibi'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['Sestamibi Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_sestamibi = preop_imaging_df[preop_imaging_df['Sestamibi Imaging Side'].isin(['Left', 'Right'])].copy()
nonlocalized_sestamibi = preop_imaging_df[preop_imaging_df[modality] == 'Non-localizing'].copy()
# total localized and non-localized sestamibi
localized_sestamibi['Calculated Concordance'] =localized_sestamibi.apply(check_concordance, axis=1)


true_positives = localized_sestamibi['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()
false_negatives = nonlocalized_sestamibi.shape[0]
false_negatives 
true_positives


112

In [29]:
sensitivity = true_positives / (true_positives + false_negatives)
print(f"Sestamibi Sensitivity: {sensitivity:.2%} ({true_positives} True Positive / {true_positives + false_negatives} total with disease)")

Sestamibi Sensitivity: 64.74% (112 True Positive / 173 total with disease)
